<a href="https://colab.research.google.com/github/ccasanoval/RLtests/blob/master/DoomV4b.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

DoomV4b

RL  = Stable Baseline 3 : PPO

ENV = Gymnasium + VizDoom

URL = https://github.com/AKapich/Reinforcement_Learning_Doom

URL = https://stable-baselines3.readthedocs.io/en/master/

In [ ]:
!pip install vizdoom
!pip install stable_baselines3

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 28.0/28.0 MB 17.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 953.9/953.9 kB 16.4 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl.metadat

In [ ]:
############### GYM ENV == VIZ DOOM ###########################################
from vizdoom import DoomGame, GameVariable
import numpy as np
from gymnasium import Env
from gymnasium.spaces import Discrete, Box
import cv2

class MyWayHomeGym(Env):
    def __init__(self, scenario, render=True, number_of_actions=3):
        self.game = DoomGame()
        self.game.load_config(f"{scenario}.cfg")

        self.game.set_window_visible(render)
        self.game.init()

        self.observation_space = Box(
            low=0, high=255, shape=(100, 160, 1), dtype=np.uint8
        )
        self.number_of_actions = number_of_actions
        self.action_space = Discrete(number_of_actions)

    def step(self, action):
        actions = np.identity(self.number_of_actions)
        reward = self.game.make_action(actions[action], 4)

        if self.game.get_state():
            state = self.game.get_state().screen_buffer
            state = self.grayscale(state)
        else:
            state = np.zeros(self.observation_space.shape)

        info = 0
        info = {"info": info}
        terminated = self.game.is_episode_finished()

        truncated = (
            self.game.is_player_dead()
            or self.game.is_player_dead()
            or self.game.is_player_dead()
        )

        return state, reward, terminated, truncated, info

    def reset(self, seed=0):
        self.game.new_episode()
        state = self.game.get_state().screen_buffer

        if self.game.get_state():
            info = self.game.get_state().game_variables[0]  # ammo
        else:
            info = 0

        return (self.grayscale(state), {"ammo": info})

    def grayscale(self, observation):
        gray = cv2.cvtColor(np.moveaxis(observation, 0, -1), cv2.COLOR_BGR2GRAY)
        resize = cv2.resize(gray, (160, 100), interpolation=cv2.INTER_CUBIC)
        state = np.reshape(resize, (100, 160, 1))
        return state

    def close(self):
        self.game.close()


In [ ]:

##################### SB3 : CALLBACK ##########################################
from stable_baselines3.common.callbacks import BaseCallback
class TrainAndLoggingCallback(BaseCallback):

    def __init__(self, check_freq, verbose=1, name="?"):
        super(TrainAndLoggingCallback, self).__init__(verbose)
        self.check_freq = check_freq
        self.name = name

    def _on_step(self):
        if self.n_calls % self.check_freq == 0:
            model_path = '{}_{}'.format(self.name, self.n_calls)
            self.model.save(model_path)
        return True



In [ ]:


modelType = "PPO"     # @param {type:"string"}
modelName = "DoomHome"+modelType # @param {type:"string"}
modelNew = True       # @param {type:"boolean"}
modelTrain = True     # @param {type:"boolean"}

print(modelName)

In [ ]:

###################### TRAIN == SB3 ###########################################
from stable_baselines3.common.evaluation import evaluate_policy
from stable_baselines3 import PPO


callback = TrainAndLoggingCallback(check_freq=10000, name=modelName)
env = MyWayHomeGym(render=False, scenario="my_way_home")

if modelNew:
    model = PPO(
        "CnnPolicy",
        env,
        verbose=1,
        seed=0,
        learning_rate=0.0001,
        n_steps=2048,
    )
else:
    model = PPO.load(modelName, env=env)

# TRAIN
if modelTrain:
    model.learn(
        total_timesteps=200000,
        callback=callback,
    )



In [ ]:
###################### TEST == SB3 ########################################
model = PPO.load(modelName, env=env)
env = env = MyWayHomeGym(render=True, scenario="my_way_home")
mean_reward, std_reward = evaluate_policy(model, env, n_eval_episodes=10)
env.close()

print(f"mean_reward:{mean_reward:.2f}")
print(f"std_reward:{std_reward:.2f}")
